In [1]:
from gs_quant.instrument import IRSwaption, IRSwap, IRCapFloor
from gs_quant.common import AggregationLevel
from gs_quant.markets.portfolio import Portfolio
from gs_quant.risk import IRVanna, IRVolga
from gs_quant.session import Environment, GsSession

In [2]:
# external users should substitute their client id and secret
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

### Solve for Vanna and Volga
Size swap to match swaption vanna

In [3]:
port = Portfolio(
    (
        IRSwaption('Pay', '5y', 'USD', expiration_date='1y', buy_sell='Sell', name='payer_swaption'),
        IRSwap(
            'Receive',
            '5y',
            'USD',
            fixed_rate='atm',
            notional_amount='=solvefor([payer_swaption].risk.IRDeltaParallel,bp)',
            name='hedge1',
        ),
    )
)
port.resolve()

In [ ]:
port.calc(IRVanna).to_frame()

In [ ]:
port.calc(IRVolga).to_frame()

Swaption (return) is non-linear (to interest change), that's why we can calculate Vanna and Volga for it, while IRSwap is linear and have it as zero.

We can also check IRVanna and IRVolga with IRCapFloor instruments, which return new options for every defined period. 

In [ ]:
cap = IRCapFloor(
    cap_floor="Cap",
    notional_currency="USD",
    notional_amount=1000,
    effective_date='2025-07-18',
    termination_date='2030-07-18',
    floating_rate_option='USD-LIBOR-BBA',
    floating_rate_designated_maturity='3m',
    floating_rate_frequency='3m',
    strike=0.015,  # 1. 5% floor rate
)
cap.resolve(in_place=False).to_dict()

In [ ]:
cap.calc(IRVanna)

In [ ]:
cap.calc(IRVanna(aggregation_level=AggregationLevel.Type))

In [ ]:
cap.calc(IRVolga)

In [ ]:
cap.calc(IRVolga(aggregation_level=AggregationLevel.Type))